In [11]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from dtreeviz.trees import *
import sklearn

def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

traindf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('/mnt/c/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
traindf['Train'] = True
testdf['Train'] = False
togeth = pd.concat([traindf,testdf])


togeth.drop(columns = 'Transported',inplace = True)


spending = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
togeth['Spending'] = togeth[spending].sum(axis = 1)


In [12]:
def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    return df
togeth = split_group_number(togeth)

def split_names(df):
    # Create new columns for first name and last name
    df['FirstName'] = df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else pd.NA)
    df['LastName'] = df['Name'].apply(lambda x: x.split()[1] if pd.notna(x) else pd.NA)
    df.drop(columns = ['Name'],inplace = True)
    return df
togeth = split_names(togeth)


def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df
togeth = split_cabin(togeth)


In [13]:
len(togeth.dropna())

9887

In [14]:
togeth.isna().sum()

PassengerId       0
HomePlanet      288
CryoSleep       310
Cabin           299
Destination     274
Age             270
VIP             296
RoomService     263
FoodCourt       289
ShoppingMall    306
Spa             284
VRDeck          268
Train             0
Spending          0
Group             0
GroupNumber       0
FirstName       294
LastName        294
CabinDeck       299
CabinSide       299
CabinNum        299
dtype: int64

**CryoSleep**

if earth and cabindeck E then not cryosleep 97% \
10 people

In [15]:
togeth.loc[(togeth.HomePlanet == 'Earth') & (togeth.CabinDeck == 'E'),'CryoSleep'] = togeth.loc[(togeth.HomePlanet == 'Earth') & (togeth.CabinDeck == 'E'),'CryoSleep'].fillna(False)

0 spending older 13 and cabindeck D 97%

In [16]:
togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'D') & (togeth.Spending == 0),'CryoSleep'] = togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'D') & (togeth.Spending == 0),'CryoSleep'].fillna(True)

0 spending older 13 and cabindeck C then Cryo True 98%

In [17]:
togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'C') & (togeth.Spending == 0),'CryoSleep'] = togeth.loc[(togeth.Age > 13) & (togeth.CabinDeck == 'C') & (togeth.Spending == 0),'CryoSleep'].fillna(True)

Europa E spending = 0 Cryosleep True = 98% but small


**Destination**

90% of people share the same destination as those with the same cabin, lastname, and cryosleep ( I think)  \ (94% if at least 3 in same cryo/cabin/lastname and majority already have same destination)